# Day 3: Gear Ratios
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [2]:
# Load Data
with open('day03_train.txt', 'r') as file:
    train_data = file.readlines()

with open('day03_test.txt', 'r') as file:
    test_data = file.readlines()

['467..114..\n',
 '...*......\n',
 '..35..633.\n',
 '......#...\n',
 '617*......\n',
 '.....+.58.\n',
 '..592.....\n',
 '......755.\n',
 '...$.*....\n',
 '.664.598..']

In [72]:
def sparsify(lines):
    fragments = {}
    prev_char = None
    for l, chars in enumerate(lines):
        for c, char in enumerate(chars):
            this_coord = (c, l)
            if char in '.\n':
                continue
            fragments[this_coord] = char
    return fragments

sparsify(train_data)         

{(0, 0): '4',
 (1, 0): '6',
 (2, 0): '7',
 (5, 0): '1',
 (6, 0): '1',
 (7, 0): '4',
 (3, 1): '*',
 (2, 2): '3',
 (3, 2): '5',
 (6, 2): '6',
 (7, 2): '3',
 (8, 2): '3',
 (6, 3): '#',
 (0, 4): '6',
 (1, 4): '1',
 (2, 4): '7',
 (3, 4): '*',
 (5, 5): '+',
 (7, 5): '5',
 (8, 5): '8',
 (2, 6): '5',
 (3, 6): '9',
 (4, 6): '2',
 (6, 7): '7',
 (7, 7): '5',
 (8, 7): '5',
 (3, 8): '$',
 (5, 8): '*',
 (1, 9): '6',
 (2, 9): '6',
 (3, 9): '4',
 (5, 9): '5',
 (6, 9): '9',
 (7, 9): '8'}

In [58]:
def sparsify_and_accumulate(lines):
    fragments = {"numbers":{}, "symbols": {}, "gears": []}
    for l, chars in enumerate(lines):
        num_start_coord = None
        for i, char in enumerate(chars):
            this_coord = (i, l)
            if char in '.\n':
                num_start_coord = None
                continue
            elif char.isdigit():
                if num_start_coord is None:
                    num_start_coord = this_coord
                    fragments["numbers"][num_start_coord] = char
                else:
                    fragments["numbers"][num_start_coord] += char
            else:
                num_start_coord = None
                fragments["symbols"][this_coord] = char
    return fragments

sparsify_and_accumulate(train_data)         

{'numbers': {(0, 0): '467',
  (5, 0): '114',
  (2, 2): '35',
  (6, 2): '633',
  (0, 4): '617',
  (7, 5): '58',
  (2, 6): '592',
  (6, 7): '755',
  (1, 9): '664',
  (5, 9): '598'},
 'symbols': {(3, 1): '*',
  (6, 3): '#',
  (3, 4): '*',
  (5, 5): '+',
  (3, 8): '$',
  (5, 8): '*'},
 'gears': []}

In [74]:
def qualify(fragments):
    numbers, symbols = fragments["numbers"], fragments["symbols"]
    qualified = []
    for c, n in numbers.items():
        left, right = c[0] - 1, c[0] + len(n)
        top, bottom = c[1] - 1, c[1] + 1
        candidates = []
        for x in range(left, right+1):
            for y in range(top, bottom+1):
                candidates.append((x,y))
        for x, y in candidates:
            if (x, y) in symbols:
                qualified.append(int(n))
                continue
    return qualified
print("Train: ",
      str(sum(qualify(sparsify_and_accumulate(train_data)))),
      "(4361)")
print("Test: ", str(sum(qualify(sparsify_and_accumulate(test_data)))))

Train:  4361 (4361)
Test:  559667


# Part 2

In [71]:
from collections import defaultdict
def find_gears(fragments):
    numbers, symbols = fragments["numbers"], fragments["symbols"]
    gears = defaultdict(list)
    for c, n in numbers.items():
        left, right = c[0] - 1, c[0] + len(n)
        top, bottom = c[1] - 1, c[1] + 1
        # candidates = list(product(range(left, right+1), range(top, bottom+1)))
        candidates = []
        for x in range(left, right+1):
            for y in range(top, bottom+1):
                candidates.append((x,y))
        # print(f"Window for {n} is {top},{left} to {bottom},{right}, yeilding {candidates}")
        for x, y in candidates:
            if (x, y) in symbols and symbols[(x, y)]=="*":
                gears[(x,y)].append(int(n))
                continue
    return gears

def get_ratios(gears):
    ratios = []
    for g in gears.values():
        if len(g) == 2:
            ratio = int(g[0]) * int(g[1])
        else:
            ratio = 0
        ratios.append(ratio)
    return ratios
    
print("Training\n",
      "Gears: ",
      find_gears(sparsify_and_accumulate(train_data)),
      "/nRatios: ",
      get_ratios(find_gears(sparsify_and_accumulate(train_data))),
      "/nTotal: ",
      sum(get_ratios(find_gears(sparsify_and_accumulate(train_data)))),
      "(should be 467835)"
     )
print("Test: ",
      sum(get_ratios(find_gears(sparsify_and_accumulate(test_data))))
     )

Training
 Gears:  defaultdict(<class 'list'>, {(3, 1): [467, 35], (3, 4): [617], (5, 8): [755, 598]}) /nRatios:  [16345, 0, 451490] /nTotal:  467835 (should be 467835)
Test:  86841457
